# Transcripciones a embeddings y almacenamiento en BBDD vectorial

In [ ]:
# ✅ Updated Code with `dotenv` and `langchain` Framework Integration
import mysql.connector
import openai
import pinecone
import os
import json
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeLangchain

# 📌 Load Environment Variables from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENVIRONMENT"))

# 📌 Connect to MySQL Database
conn = mysql.connector.connect(
    host=os.getenv("MYSQL_HOST"),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DATABASE")
)
cursor = conn.cursor()

# 📌 Fetch Transcriptions from MySQL
cursor.execute("SELECT call_id, transcription FROM transcriptions WHERE status IN ('Success', 'Failed')")
transcriptions = cursor.fetchall()

# 📌 Initialize Langchain with OpenAI Embeddings and Pinecone
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
pinecone_store = PineconeLangchain(index_name=os.getenv("PINECONE_INDEX"), embedding=embeddings)

# 📌 Process Each Transcription and Store in Pinecone via Langchain
for call_id, transcription_json in transcriptions:
    transcription_data = json.loads(transcription_json)
    for i, entry in enumerate(transcription_data):
        speaker = entry["speaker"]
        text = entry["text"]

        vector_id = f"{call_id}_{i+1}"

        # Add document to Langchain Pinecone store with metadata
        pinecone_store.add_texts(
            texts=[text],
            metadatas=[{"call_id": call_id, "speaker": speaker, "text": text}],
            ids=[vector_id]
        )
        print(f"✅ Stored via Langchain: {vector_id} | {speaker} | {text}")

# 📌 Close MySQL Connection
cursor.close()
conn.close()
print("🚀 All transcriptions stored in Pinecone via Langchain!")